## QTL Ensembl VEP

**Created**: 10 June 2022

## Environment

In [1]:
library(tidyverse)
library(data.table)

setwd("~/eQTL_pQTL_Characterization/")

source("03_Functional_Interpretation/scripts/utils/ggplot_theme.R")

── Attaching packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose




## Load Data

In [2]:
vep <- fread("~/gains_team282/epigenetics/vep/ensembl_vep/qtl_vep_output.tsv", sep="\t", na.strings="") %>%
    as.data.frame()

In [3]:
head(vep)

,CHROM,POS,ID,REF,ALT,Allele,Consequence,IMPACT,SYMBOL,Gene,⋯,REVEL,SpliceAI_pred_DP_AG,SpliceAI_pred_DP_AL,SpliceAI_pred_DP_DG,SpliceAI_pred_DP_DL,SpliceAI_pred_DS_AG,SpliceAI_pred_DS_AL,SpliceAI_pred_DS_DG,SpliceAI_pred_DS_DL,SpliceAI_pred_SYMBOL
,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,1,817341,rs3131972,G,A,A,upstream_gene_variant,MODIFIER,FAM87B,ENSG00000177757,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,1,817341,rs3131972,G,A,A,intron_variant&non_coding_transcript_variant,MODIFIER,NA,ENSG00000230092,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,1,817341,rs3131972,G,A,A,intron_variant&non_coding_transcript_variant,MODIFIER,NA,ENSG00000230021,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,1,817341,rs3131972,G,A,A,intron_variant&non_coding_transcript_variant,MODIFIER,NA,ENSG00000230021,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,1,817341,rs3131972,G,A,A,regulatory_region_variant,MODIFIER,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,817341,rs3131972,G,A,A,TF_binding_site_variant,MODIFIER,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [4]:
colnames(vep)

[1] "CHROM"                              "POS"                               
  [3] "ID"                                 "REF"                               
  [5] "ALT"                                "Allele"                            
  [7] "Consequence"                        "IMPACT"                            
  [9] "SYMBOL"                             "Gene"                              
 [11] "Feature_type"                       "Feature"                           
 [13] "BIOTYPE"                            "EXON"                              
 [15] "INTRON"                             "HGVSc"                             
 [17] "HGVSp"                              "cDNA_position"                     
 [19] "CDS_position"                       "Protein_position"                  
 [21] "Amino_acids"                        "Codons"                            
 [23] "Existing_variation"                 "ALLELE_NUM"                        
 [25] "DISTANCE"                           "STRAND"                            
 [27] "FLAGS"                              "VARIANT_CLASS"                     
 [29] "SYMBOL_SOURCE"                      "HGNC_ID"                           
 [31] "CANONICAL"                          "MANE_SELECT"                       
 [33] "MANE_PLUS_CLINICAL"                 "TSL"                               
 [35] "APPRIS"                             "CCDS"                              
 [37] "ENSP"                               "SWISSPROT"                         
 [39] "TREMBL"                             "UNIPARC"                           
 [41] "UNIPROT_ISOFORM"                    "GENE_PHENO"                        
 [43] "SIFT"                               "PolyPhen"                          
 [45] "DOMAINS"                            "miRNA"                             
 [47] "HGVS_OFFSET"                        "AF"                                
 [49] "AFR_AF"                             "AMR_AF"                            
 [51] "EAS_AF"                             "EUR_AF"                            
 [53] "SAS_AF"                             "AA_AF"                             
 [55] "EA_AF"                              "gnomAD_AF"                         
 [57] "gnomAD_AFR_AF"                      "gnomAD_AMR_AF"                     
 [59] "gnomAD_ASJ_AF"                      "gnomAD_EAS_AF"                     
 [61] "gnomAD_FIN_AF"                      "gnomAD_NFE_AF"                     
 [63] "gnomAD_OTH_AF"                      "gnomAD_SAS_AF"                     
 [65] "MAX_AF"                             "MAX_AF_POPS"                       
 [67] "CLIN_SIG"                           "SOMATIC"                           
 [69] "PHENO"                              "PUBMED"                            
 [71] "MOTIF_NAME"                         "MOTIF_POS"                         
 [73] "HIGH_INF_POS"                       "MOTIF_SCORE_CHANGE"                
 [75] "TRANSCRIPTION_FACTORS"              "SpliceRegion"                      
 [77] "GeneSplicer"                        "existing_InFrame_oORFs"            
 [79] "existing_OutOfFrame_oORFs"          "existing_uORFs"                    
 [81] "five_prime_UTR_variant_annotation"  "five_prime_UTR_variant_consequence"
 [83] "CADD_PHRED"                         "CADD_RAW"                          
 [85] "Ensembl_transcriptid"               "LRT_pred"                          
 [87] "MutationTaster_pred"                "Polyphen2_HDIV_pred"               
 [89] "Polyphen2_HVAR_pred"                "SIFT_pred"                         
 [91] "Uniprot_acc"                        "VEP_canonical"                     
 [93] "DisGeNET_PMID"                      "DisGeNET_SCORE"                    
 [95] "PHENOTYPES"                         "Conservation"                      
 [97] "LoF"                                "LoF_filter"                        
 [99] "LoF_flags"                          "LoF_info"                          
[1

In [22]:
gene.info <- read.table("/nfs/team282/data/gains_team282/gene_info_864_20412_hla.txt") %>%
    dplyr::select(gene_id, gene_name)

In [23]:
cis.eqtl <- read.table("~/gains_team282/eqtl/cisresults/eigenMT/ciseqtl_eigenMT_corrected.txt") %>%
    dplyr::filter(Sig) %>%
    merge(gene.info, ., by.x="gene_id", by.y="gene")

In [57]:
head(cis.eqtl)

,gene_id,gene_name,snps,statistic,pvalue,beta,se,chr,SNPpos,TSS,BF,TESTS,BF.FDR,Sig,threshold
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<int>,<dbl>,<lgl>,<dbl>
1,ENSG00000000457,SCYL3,rs10753794,7.037066,5.192595e-12,0.04985873,0.007085159,1,169776143,169894267,3.608853e-09,695,1.308708e-08,TRUE,3.879445e-05
2,ENSG00000000460,C1orf112,rs12410806,-5.942351,5.426681e-09,-0.07212698,0.012137784,1,169749095,169662007,4.032024e-06,743,1.160666e-05,TRUE,3.628822e-05
3,ENSG00000000971,CFH,rs12406047,4.511592,7.515235e-06,0.17135374,0.037980769,1,196708768,196652043,2.818213e-03,375,6.073985e-03,TRUE,7.189906e-05
4,ENSG00000001036,FUCA2,rs2496591,5.771056,1.272071e-08,0.05556388,0.009628027,6,143416006,143511720,1.017657e-05,800,2.830516e-05,TRUE,3.370268e-05
5,ENSG00000001167,NFYA,rs72856259,7.552123,1.978551e-13,0.11980933,0.015864325,6,41100761,41072945,1.889516e-10,955,7.447584e-10,TRUE,2.823261e-05
6,ENSG00000001460,STPG1,rs6687210,5.266595,1.949343e-07,0.09241162,0.017546749,1,24405200,24416934,9.668744e-05,496,2.450046e-04,TRUE,5.435916e-05


In [25]:
mqtl <- readRDS("~/gains_team282/nikhil/expression/eigengene_sva/mqtl.RDS") %>%
    dplyr::group_by(module, qtl.locus) %>%
    dplyr::slice_min(p, n=1, with_ties=F)

In [27]:
head(mqtl)

snp,beta,se,t,p,module.qtl,module,pc,qtl.locus,qtl.locus.chr,qtl.locus.start,qtl.locus.end
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
rs16866400,-0.01081391,0.0016948840,-6.380325,3.197894e-10,ME_100_4-2:177556567-179584183,ME_100,4,2:177556567-179584183,2,177556567,179584183
rs582717,-0.02334898,0.0014724757,-15.856953,7.745374e-48,ME_101_2-1:24243590-26454176,ME_101,2,1:24243590-26454176,1,24243590,26454176
rs1131017,0.02409859,0.0008054830,29.918193,7.188768e-121,ME_101_3-12:54985643-57100037,ME_101,3,12:54985643-57100037,12,54985643,57100037
rs12442141,0.04014834,0.0033260563,12.070853,1.789047e-30,ME_101_5-15:30215896-32681425,ME_101,5,15:30215896-32681425,15,30215896,32681425
rs629329,0.02986714,0.0014708263,20.306367,5.848637e-71,ME_102_1-17:17145294-19676395,ME_102,1,17:17145294-19676395,17,17145294,19676395
rs4761234,0.01989013,0.0005288609,37.609373,8.488460e-164,ME_103_3-12:68275923-70376531,ME_103,3,12:68275923-70376531,12,68275923,70376531


In [31]:
cis.pqtl <- readRDS("/nfs/users/nfs_n/nm18/gains_team282/proteomics/pqtl/pqtl_ms2019/cis_pqtl_all.RDS") %>%
    dplyr::filter(pQTL_pval < (5e-8) / 269) %>%
    dplyr::group_by(Gene) %>%
    dplyr::slice_min(pQTL_pval, n=1)

In [32]:
trans.pqtl <- readRDS("/nfs/users/nfs_n/nm18/gains_team282/proteomics/pqtl/pqtl_ms2019/trans_pqtl_all.RDS") %>%
    dplyr::filter(pQTL_pval < (5e-8) / 269) %>%
    dplyr::group_by(pQTL_Protein, pQTL_Locus) %>%
    dplyr::slice_min(pQTL_pval, n=1)

In [33]:
head(cis.pqtl)

SNP,Chr,Position,pQTL_beta,pQTL_SE,pQTL_t,pQTL_pval,pQTL_Protein,Gene
<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
rs529541,1,196750586,-0.24905935,0.01325800,-18.785585,1.642588e-67,P08603,ENSG00000000971
rs854572,7,95325384,0.18402863,0.01585951,11.603674,3.259335e-29,P27169,ENSG00000005421
rs6073958,20,45923216,-0.14494261,0.01847291,-7.846223,8.488455e-15,P55058,ENSG00000100979
rs7815785,8,17872829,-0.22225713,0.02073768,-10.717549,2.077001e-25,Q08830,ENSG00000104760
rs1648722,3,186731200,-0.06974747,0.01034678,-6.740986,2.709794e-11,P01042,ENSG00000113889
rs1656925,3,186731334,-0.06974747,0.01034678,-6.740986,2.709794e-11,P01042,ENSG00000113889


In [34]:
head(trans.pqtl)

SNP,Chr,Position,pQTL_beta,pQTL_SE,pQTL_t,pQTL_pval,pQTL_Protein,pQTL_Locus
<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
rs11846959,14,94379979,-0.3065649,0.01614662,-18.986322,3.309735e-69,P00751,pQTL_1
rs11846959,14,94379979,-0.2137874,0.01793464,-11.920362,6.923034e-31,P02787,pQTL_1
rs9302635,16,72110275,0.1145890,0.01466176,7.815500,1.200982e-14,P10909,pQTL_1
rs9302635,16,72110275,0.1319606,0.01429908,9.228609,1.454631e-19,P19827,pQTL_1
rs16917722,8,95461382,-0.4991650,0.06949047,-7.183215,1.673847e-12,P35858,pQTL_1
rs76368529,8,95464005,-0.4991650,0.06949047,-7.183215,1.673847e-12,P35858,pQTL_1


## Consequence and Impact

In [79]:
cis.eqtl.vep <- merge(vep, cis.eqtl, by.x="ID", by.y="snps") %>%
    dplyr::select(
        gene_id, CHROM, POS, ID, REF, ALT, Allele,
        Gene, Feature_type, Feature,
        Consequence, IMPACT,
        BIOTYPE,
        MOTIF_NAME, TRANSCRIPTION_FACTORS,
        CADD_PHRED, CADD_RAW, PolyPhen
    )

In [71]:
head(cis.eqtl.vep)

,gene_id,CHROM,POS,ID,REF,ALT,Allele,Gene,Feature_type,Feature,Consequence,IMPACT,BIOTYPE,MOTIF_NAME,TRANSCRIPTION_FACTORS,CADD_PHRED,CADD_RAW,PolyPhen
,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
1,ENSG00000272416,5,54079901,5.54079901_A_C,C,A,A,ENSG00000185305,Transcript,ENST00000504924,intron_variant,MODIFIER,protein_coding,NA,NA,1.589,0.037598,NA
2,ENSG00000272416,5,54079901,5.54079901_A_C,C,A,A,ENSG00000185305,Transcript,ENST00000502271,intron_variant,MODIFIER,protein_coding,NA,NA,1.589,0.037598,NA
3,ENSG00000272416,5,54079901,5.54079901_A_C,C,A,A,ENSG00000185305,Transcript,ENST00000510591,intron_variant&non_coding_transcript_variant,MODIFIER,processed_transcript,NA,NA,1.589,0.037598,NA
4,ENSG00000272416,5,54079901,5.54079901_A_C,C,A,A,ENSG00000265421,Transcript,ENST00000581837,upstream_gene_variant,MODIFIER,miRNA,NA,NA,1.589,0.037598,NA
5,ENSG00000272416,5,54079901,5.54079901_A_C,A,C,C,ENSG00000185305,Transcript,ENST00000504924,intron_variant,MODIFIER,protein_coding,NA,NA,NA,NA,NA
6,ENSG00000272416,5,54079901,5.54079901_A_C,C,A,A,ENSG00000185305,Transcript,ENST00000620747,intron_variant,MODIFIER,protein_coding,NA,NA,1.589,0.037598,NA


In [81]:
cis.eqtl.vep %>%
    dplyr::select(-REF, -ALT, -Allele) %>%
    unique() %>%
    dplyr::count(gene_id, Consequence) %>%
    dplyr::filter(Consequence == "missense_variant&NMD_transcript_variant")

gene_id,Consequence,n
<chr>,<chr>,<int>
ENSG00000022976,missense_variant&NMD_transcript_variant,2
ENSG00000059691,missense_variant&NMD_transcript_variant,6
ENSG00000072694,missense_variant&NMD_transcript_variant,6
ENSG00000087263,missense_variant&NMD_transcript_variant,44
ENSG00000099330,missense_variant&NMD_transcript_variant,4
ENSG00000099337,missense_variant&NMD_transcript_variant,6
ENSG00000100938,missense_variant&NMD_transcript_variant,2
ENSG00000101639,missense_variant&NMD_transcript_variant,4
ENSG00000104884,missense_variant&NMD_transcript_variant,2


In [90]:
cis.eqtl.vep %>%
    dplyr::select(-REF, -ALT, -Allele, -CADD_PHRED, -CADD_RAW, -PolyPhen) %>%
    unique() %>%
    dplyr::count(gene_id, Consequence) %>%
    dplyr::group_by(Consequence) %>%
    dplyr::summarize(n=sum(n), egenes=n())

Consequence,n,egenes
<chr>,<int>,<int>
3_prime_UTR_variant,996,524
3_prime_UTR_variant&NMD_transcript_variant,363,205
5_prime_UTR_variant,501,227
5_prime_UTR_variant&NMD_transcript_variant,72,44
downstream_gene_variant,14899,4175
intergenic_variant,1035,1035
intron_variant,22869,5871
intron_variant&NMD_transcript_variant,5618,2444
intron_variant&non_coding_transcript_variant,12813,4488


In [87]:
t(test)

gene_id,ENSG00000059691,ENSG00000059691,ENSG00000059691
CHROM,4,4,4
POS,151760894,151760894,151760894
ID,rs11556167,rs11556167,rs11556167
Gene,ENSG00000059691,ENSG00000059691,ENSG00000059691
Feature_type,Transcript,Transcript,Transcript
Feature,ENST00000511538,ENST00000503160,ENST00000515884
Consequence,missense_variant&NMD_transcript_variant,missense_variant&NMD_transcript_variant,missense_variant&NMD_transcript_variant
IMPACT,MODERATE,MODERATE,MODERATE
BIOTYPE,nonsense_mediated_decay,nonsense_mediated_decay,nonsense_mediated_decay
MOTIF_NAME,NA,NA,NA
